# Building the Digital Library (15, June 2024)

The aim of this notebook is to used improved chunking methods and allow for document/title based retreival

In [1]:
!pip install -qU \
    datasets==2.19.0

In [4]:
import subprocess
import os
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

In [2]:
from datasets import load_dataset

dataset = load_dataset("jamescalam/ai-arxiv2", split="train")
dataset

Generating train split:   0%|          | 0/2673 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'content', 'references'],
    num_rows: 2673
})

In [8]:
from sentence_transformers import SentenceTransformer
model_name = "all-MiniLM-L6-v2"  # This is a small and efficient model for text embeddings
encoder = SentenceTransformer(model_name)


In [6]:
HuggingFaceEmbeddingsModel = 'BAAI/bge-base-en-v1.5'
encoder = HuggingFaceEmbeddings(model_name=HuggingFaceEmbeddingsModel)

In [10]:
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger

logger.setLevel("WARNING")  # reduce logs from splitter

splitter = RollingWindowSplitter(
    encoder=encoder,
    dynamic_threshold=True,
    min_split_tokens=100,
    max_split_tokens=500,
    window_size=2,
    plot_splits=True,  # set this to true to visualize chunking
    enable_statistics=True  # to print chunking stats
)

/tmp/ipykernel_3630655/2902977683.py:6: UserWarning: Splitters are being deprecated. They have moved to their own package. Please migrate to the `semantic-chunkers` package. More information can be found at:
https://github.com/aurelio-labs/semantic-chunkers
  splitter = RollingWindowSplitter(


ValidationError: 1 validation error for RollingWindowSplitter
encoder
  value is not a valid dict (type=type_error.dict)

In [12]:
type(encoder)

sentence_transformers.SentenceTransformer.SentenceTransformer

In [11]:
help(RollingWindowSplitter)

Help on class RollingWindowSplitter in module semantic_router.splitters.rolling_window:

class RollingWindowSplitter(semantic_router.splitters.base.BaseSplitter)
 |  RollingWindowSplitter(encoder: semantic_router.encoders.base.BaseEncoder, name='rolling_window_splitter', threshold_adjustment=0.01, dynamic_threshold: bool = True, window_size=5, min_split_tokens=100, max_split_tokens=300, split_tokens_tolerance=10, plot_splits=False, enable_statistics=False) -> None
 |  
 |  Method resolution order:
 |      RollingWindowSplitter
 |      semantic_router.splitters.base.BaseSplitter
 |      pydantic.v1.main.BaseModel
 |      pydantic.v1.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, docs: List[str]) -> List[semantic_router.schema.DocumentSplit]
 |      Split documents into smaller chunks based on semantic similarity.
 |      
 |      :param docs: list of text documents to be split, if only wanted to
 |          split a single document, pa

In [2]:
!pip show semanitc-router